# Regrid Chassignet & Xu Salinity File to Gridtools NWA25

In [1]:
import xesmf as xe
import xarray as xr
import numpy as np

In [2]:
old_ds = xr.open_dataset('sss_mom6.nc', decode_times='False')
old_ds

<xarray.Dataset>
Dimensions:    (MT: 12, X: 1678, Y: 1844)
Coordinates:
    Date       (MT) float64 ...
    Latitude   (Y, X) float32 ...
    Longitude  (Y, X) float32 ...
  * MT         (MT) datetime64[ns] 1904-01-16 ... 1904-12-16T12:00:00
  * X          (X) int32 2349 2350 2351 2352 2353 ... 4022 4023 4024 4025 4026
  * Y          (Y) int32 1141 1142 1143 1144 1145 ... 2980 2981 2982 2983 2984
Data variables:
    SSS        (MT, Y, X) float32 ...
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM
    source:       HYCOM forcing file
    history:      Wed Jul 17 18:38:07 2019: ncks -d X,2348,4025 -d Y,1140,298...
    NCO:          netCDF Operators version 4.7.7 (Homepage = http://nco.sf.ne...

In [3]:
old_ds = old_ds.rename({"Longitude": "lon", "Latitude": "lat"})
old_ds

<xarray.Dataset>
Dimensions:  (MT: 12, X: 1678, Y: 1844)
Coordinates:
    Date     (MT) float64 1.904e+07 1.904e+07 1.904e+07 ... 1.904e+07 1.904e+07
    lat      (Y, X) float32 ...
    lon      (Y, X) float32 ...
  * MT       (MT) datetime64[ns] 1904-01-16 ... 1904-12-16T12:00:00
  * X        (X) int32 2349 2350 2351 2352 2353 ... 4022 4023 4024 4025 4026
  * Y        (Y) int32 1141 1142 1143 1144 1145 ... 2980 2981 2982 2983 2984
Data variables:
    SSS      (MT, Y, X) float32 ...
Attributes:
    Conventions:  CF-1.0
    title:        HYCOM
    source:       HYCOM forcing file
    history:      Wed Jul 17 18:38:07 2019: ncks -d X,2348,4025 -d Y,1140,298...
    NCO:          netCDF Operators version 4.7.7 (Homepage = http://nco.sf.ne...

In [4]:
new_ds = xr.open_dataset('ocean_mask.nc')
new_ds

<xarray.Dataset>
Dimensions:  (nx: 1500, ny: 1250)
Dimensions without coordinates: nx, ny
Data variables:
    mask     (ny, nx) float64 ...
    x        (ny, nx) float64 ...
    y        (ny, nx) float64 ...
Attributes:
    grid_version:  0.2
    code_version:  ESMG/gridtools: ['heads/main-0-gdd0ea25632-dirty', '']
    history:       2023-01-16: gridtools.grid.mom6.convert_ROMS_to_MOM6 + kwargs

In [5]:
new_ds = new_ds.rename({"x": "lon", "y": "lat"})
new_ds

<xarray.Dataset>
Dimensions:  (nx: 1500, ny: 1250)
Dimensions without coordinates: nx, ny
Data variables:
    mask     (ny, nx) float64 ...
    lon      (ny, nx) float64 ...
    lat      (ny, nx) float64 ...
Attributes:
    grid_version:  0.2
    code_version:  ESMG/gridtools: ['heads/main-0-gdd0ea25632-dirty', '']
    history:       2023-01-16: gridtools.grid.mom6.convert_ROMS_to_MOM6 + kwargs

In [6]:
lon_centers = new_ds.lon.values
lon_corners = 0.5 * (lon_centers[:-1, :-1] + lon_centers[1:, 1:])
nrows, ncols = lon_centers.shape
lon_b = np.zeros((nrows+1, ncols+1))
lon_b[1:-1, 1:-1] = lon_corners

# Estimate the values on the left and right columns
lon_b[1:-1, 0] = 2 * lon_centers[:-1, 0] - lon_b[1:-1, 1]
lon_b[1:-1, -1] = 2 * lon_centers[:-1, -1] - lon_b[1:-1, -2]

# Estimate the values on the top and bottom rows
lon_b[0, 1:-1] = lon_centers[0, :-1]
lon_b[-1, 1:-1] = lon_centers[-1, :-1]

lon_b[0, 0] = 2 * lon_centers[0, 0] - lon_b[1, 1]
lon_b[0, -1] = 2 * lon_centers[0, -1] - lon_b[1, -2]
lon_b[-1, 0] = 2 * lon_centers[-1, 0] - lon_b[-2, 1]
lon_b[-1, -1] = 2 * lon_centers[-1, -1] - lon_b[-2, -2]

In [7]:
lat_centers = new_ds.lat.values
lat_corners = 0.5 * (lat_centers[:-1, :-1] + lat_centers[1:, 1:])
nrows, ncols = lat_centers.shape
lat_b = np.zeros((nrows+1, ncols+1))
lat_b[1:-1, 1:-1] = lat_corners

# Estimate the values on the left and right columns
lat_b[1:-1, 0] = 2 * lat_centers[:-1, 0] - lat_b[1:-1, 1]
lat_b[1:-1, -1] = 2 * lat_centers[:-1, -1] - lat_b[1:-1, -2]

# Estimate the values on the top and bottom rows
lat_b[0, 1:-1] = lat_centers[0, :-1]
lat_b[-1, 1:-1] = lat_centers[-1, :-1]

lat_b[0, 0] = 2 * lat_centers[0, 0] - lat_b[1, 1]
lat_b[0, -1] = 2 * lat_centers[0, -1] - lat_b[1, -2]
lat_b[-1, 0] = 2 * lat_centers[-1, 0] - lat_b[-2, 1]
lat_b[-1, -1] = 2 * lat_centers[-1, -1] - lat_b[-2, -2]

In [204]:
nxp = np.append(new_ds['nx'].values, len(new_ds['nx'].values))
nyp = np.append(new_ds['ny'].values, len(new_ds['ny'].values))


In [18]:
new_ds = xr.open_dataset('ocean_mask.nc')
lat = new_ds.y
lon = new_ds.x
# add nxp and nyp dimensions for the lat/lon corners to latch onto
new_ds = new_ds.expand_dims({'nyp':(len(new_ds.ny) + 1)})
new_ds = new_ds.expand_dims({'nxp':(len(new_ds.nx) + 1)})
new_ds

<xarray.Dataset>
Dimensions:  (nx: 1500, nxp: 1501, ny: 1250, nyp: 1251)
Dimensions without coordinates: nx, nxp, ny, nyp
Data variables:
    mask     (nxp, nyp, ny, nx) float64 0.0 0.0 0.0 0.0 0.0 ... 1.0 1.0 1.0 1.0
    x        (nxp, nyp, ny, nx) float64 -98.98 -98.94 -98.9 ... -39.06 -39.02
    y        (nxp, nyp, ny, nx) float64 4.02 4.02 4.02 ... 53.98 53.98 53.98
Attributes:
    grid_version:  0.2
    code_version:  ESMG/gridtools: ['heads/main-0-gdd0ea25632-dirty', '']
    history:       2023-01-16: gridtools.grid.mom6.convert_ROMS_to_MOM6 + kwargs

In [19]:
new_ds = new_ds.drop_vars('mask')
new_ds = new_ds.drop_vars('x')
new_ds = new_ds.drop_vars('y')
new_ds['lat'] = (('ny', 'nx'), lat)
new_ds['lon'] = (('ny', 'nx'), lon)
new_ds

<xarray.Dataset>
Dimensions:  (nx: 1500, ny: 1250)
Dimensions without coordinates: nx, ny
Data variables:
    lat      (ny, nx) float64 4.02 4.02 4.02 4.02 ... 53.98 53.98 53.98 53.98
    lon      (ny, nx) float64 -98.98 -98.94 -98.9 -98.86 ... -39.1 -39.06 -39.02
Attributes:
    grid_version:  0.2
    code_version:  ESMG/gridtools: ['heads/main-0-gdd0ea25632-dirty', '']
    history:       2023-01-16: gridtools.grid.mom6.convert_ROMS_to_MOM6 + kwargs

In [20]:
new_ds = new_ds.assign(lon_b=xr.DataArray(lon_b, dims=("nyp", "nxp")))
new_ds = new_ds.assign(lat_b=xr.DataArray(lat_b, dims=("nyp", "nxp")))

In [21]:
new_ds

<xarray.Dataset>
Dimensions:  (nx: 1500, nxp: 1501, ny: 1250, nyp: 1251)
Dimensions without coordinates: nx, nxp, ny, nyp
Data variables:
    lat      (ny, nx) float64 4.02 4.02 4.02 4.02 ... 53.98 53.98 53.98 53.98
    lon      (ny, nx) float64 -98.98 -98.94 -98.9 -98.86 ... -39.1 -39.06 -39.02
    lon_b    (nyp, nxp) float64 -99.0 -98.98 -98.94 -98.9 ... -39.1 -39.06 -39.0
    lat_b    (nyp, nxp) float64 4.0 4.02 4.02 4.02 ... 53.98 53.98 53.98 54.0
Attributes:
    grid_version:  0.2
    code_version:  ESMG/gridtools: ['heads/main-0-gdd0ea25632-dirty', '']
    history:       2023-01-16: gridtools.grid.mom6.convert_ROMS_to_MOM6 + kwargs

In [ ]:
regridder = xe.Regridder(old_ds, new_ds, "bilinear")
regridder  # print basic regridder information.



Create weight file: bilinear_1844x1678_1250x1500.nc
